# 2020 US Election Progress Analysis

https://www.theguardian.com/us-news/ng-interactive/2020/nov/07/us-election-2020-live-results-donald-trump-joe-biden-presidential-votes-pennsylvania-georgia-arizona-nevada

In [1]:
import requests
import pandas as pd
import datetime
import dateutil.parser
import json
import re
import glob
import sys
import locale
locale.setlocale(locale.LC_ALL, '')  # Use '' for auto, or force e.g. to 'en_US.UTF-8'

# gimme 100% width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

def format_percent(x):
    return f'{x*100:.2f}'

def flattened(l):
    return [item for sublist in l for item in sublist]

NAME_BY_ACRONYM = {
    'AL': 'Alabama',        'AK': 'Alaska',        'AZ': 'Arizona',    'AR': 'Arkansas',             'CA': 'California',
    'CO': 'Colorado',       'CT': 'Connecticut',   'DE': 'Delaware',   'DC': 'District of Columbia', 'FL': 'Florida',
    'GA': 'Georgia',        'HI': 'Hawaii',        'ID': 'Idaho',      'IL': 'Illinois',             'IN': 'Indiana',
    'IA': 'Iowa',           'KS': 'Kansas',        'KY': 'Kentucky',   'LA': 'Louisiana',            'ME': 'Maine',
    'MD': 'Maryland',       'MA': 'Massachusetts', 'MI': 'Michigan',   'MN': 'Minnesota',            'MS': 'Mississippi',
    'MO': 'Missouri',       'MT': 'Montana',       'NE': 'Nebraska',   'NV': 'Nevada',               'NH': 'New Hampshire',
    'NJ': 'New Jersey',     'NM': 'New Mexico',    'NY': 'New York',   'NC': 'North Carolina',       'ND': 'North Dakota',
    'OH': 'Ohio',           'OK': 'Oklahoma',      'OR': 'Oregon',     'PA': 'Pennsylvania',         'RI': 'Rhode Island',
    'SC': 'South Carolina', 'SD': 'South Dakota',  'TN': 'Tennessee',  'TX': 'Texas',    'UT':       'Utah',
    'VT': 'Vermont',        'VA': 'Virginia',      'WA': 'Washington', 'WV': 'West Virginia',        'WI': 'Wisconsin',
    'WY': 'Wyoming',        'PR': 'Puerto Rico',
}

def load(file_name=None):
    if file_name:
        with open(file_name) as f:
            return json.load(f)
    
    # Load dataset
    try:
        # From the Guardian on the Web...
        last_update = requests.get('https://interactive.guim.co.uk/2020/11/us-general-election-data/prod/last_updated.json').json()
        last_time = last_update['time']
        url = f'https://interactive.guim.co.uk/2020/11/us-general-election-data/prod/data-out/{last_time}/president_details.json'
        print(last_time)
        data = requests.get(url).json()
        fname = 'data-cache--' + re.compile('[^0-9a-zA-Z-]').sub('.', last_time) + '.json'
        with open(fname, 'w') as f: json.dump(data, f, indent='  ', sort_keys=True)
    except Exception as e:
        # ... or cached
        print(e, file=sys.stderr)
        names = sorted(glob.glob('data-cache--*.json'))
        print('Found cache:', names)
        name = names[-1]
        print('Loading:', name)
        with open(name) as f: data = json.load(f)
    return data

def compute(data):
    electors_by_candidate = {}
    electors_by_candidate_may = {}
    electors_by_state = {}

    new_data = {}
    new_data_short = {}

    n = 0

    # Compute electors by state
    for acronym, state_data in data.items():
        if acronym == 'US': continue
        electors = max( c['electWon'] for c in state_data['candidates'])
        # Some return zero, fix that
        electors_by_state[acronym] = {'AK': 6, 'GA': 16}.get(acronym, electors)

    # Analyze each state
    for acronym, state_data in data.items():
        if acronym == 'US': continue
        n += 1
        extra = {}
        extra_short = {}
        electors = electors_by_state[acronym]

        # Analyze each field
        for k, v in state_data.items():
            if k == 'declarationTime' and v:
                # make it shorter
                dt = dateutil.parser.parse(v)
                state_data[k] = datetime.datetime.strftime(dt, '%b %d %H:%M')
            if k == 'candidates':
                candidate = {c['name'].split()[-1]:c for c in v}
                if n and n < 4:
                    pass #print();print(candidate)
                # compute votes
                nb_counted = sum([c['votes'] for c in v])
                max_votes = int(state_data['totalVotes'] / state_data['reporting'] * 100.)
                rem_votes = max_votes - nb_counted
                # general fields
                #extra['electors'] = electors
                extra['% counted']   = extra_short['% counted']   = state_data['reporting']
                extra['grand total'] = extra_short['grand total'] = max_votes
                extra['uncounted']   = extra_short['uncounted']   = rem_votes
                # per-candidate votes
                extra['- SCORE -'] = ''
                for name, cdata in candidate.items():
                    val = cdata['votes'] / nb_counted
                    extra[name]       = extra_short[name]       = cdata['votes']
                for name, cdata in candidate.items():
                    val = cdata['votes'] / nb_counted
                    extra[name + '%']       = format_percent(val)
                    extra_short[name + '%'] = val
                # winner
                winner = sorted([(cdata['votes'], name) for name, cdata in candidate.items()])[-1][1]
                extra['winner'] = winner
                # compute "maybe" winner: who could win, if getting all remaining votes
                extra['- MAYBE -'] = ''
                cand2 = {c['votes']:c['name'].split()[-1] for c in v}
                possibilities = [
                    [[votes2 + (rem_votes if name2!=winner else 0), name2] for votes2, name2 in cand2.items()]
                    for votes1, name1 in cand2.items()]
                winner_may_votes, winner_may = sorted(flattened(possibilities))[-1]
                for name, cdata in candidate.items():
                    val = (cdata['votes']+rem_votes) / (nb_counted+rem_votes)
                    extra[name + '%may'] = format_percent(val) if name != winner else f'({format_percent(val)})'
                extra['may'] = winner_may if winner_may != winner else ''
                #extra['may votes'] = winner_may_votes
                #extra['may %'] = format_percent(winner_may_votes / max_votes)

        # accumulate won electors
        electors_by_candidate.setdefault(winner, 0)
        electors_by_candidate[winner] += electors
        electors_by_candidate_may.setdefault(winner_may, 0)
        electors_by_candidate_may[winner_may] += electors

        # format ints
        for k, v in extra.items():
            if v.__class__ is int:
                extra[k] = f'{v:n}'
        # keep new data and format ints
        new_data[acronym] = extra
        new_data_short[acronym] = extra_short

    electors_by_candidate = [{'Candidate': cand, 'Electors': electors} for cand, electors in electors_by_candidate.items()]
    electors_by_candidate_may = [{'Candidate maybe': cand, 'Electors': electors} for cand, electors in electors_by_candidate_may.items()]
    electors_by_state = {acronym:{'Name': NAME_BY_ACRONYM[acronym], 'Electors': nb} for acronym, nb in electors_by_state.items()}
    
    return new_data, new_data_short, electors_by_candidate, electors_by_candidate_may, electors_by_state

def dic_dict_sub(d1, d2):
    def fmt(v):
        if v.__class__ == int:
            return f'{v:n}'
        elif v.__class__ == float:
            return f'{v*100:.2f}'
        else:
            return v
        
    return { k1:{k2:fmt(v2-d2[k1][k2]) for k2, v2 in v1.items()} for k1, v1 in d1.items()}

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)

# 1. Results as of Sat 7 Nov

In [2]:
data = load('data-cache--2020-11-07T17.14.12.533.00.00.json')

new_data, new_data_short_0, electors_by_candidate, electors_by_candidate_may, electors_by_state = compute(data)
pd.DataFrame(new_data)

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,50,99,99,90,66,95,99,90,99,99,99,96,99,99,92,99,99,99,95,99,86,99,99,99,99,99,99,99,99,99,99,77,99,92,78,96,99,98,99,99,99,94,99,98,93,99,92,85,99,99,97
grand total,381'940,2'313'155,1'220'137,3'659'095,21'489'998,3'340'132,1'788'948,319'751,507'418,11'155'090,5'033'023,597'764,1'700'530,875'985,5'950'636,3'054'370,1'346'982,2'155'455,2'260'329,3'564'678,2'859'958,821'532,5'575'096,3'301'777,3'038'125,1'144'988,606'747,5'513'657,362'294,945'036,809'828,4'486'316,926'419,1'408'439,8'503'584,6'001'302,1'574'370,2'335'332,6'815'155,510'304,2'538'256,449'571,3'076'244,11'470'733,1'397'625,4'419'261,397'220,4'608'967,3'322'701,793'051,283'373
uncounted,194'148,23'131,33'386,365'909,7'446'895,200'994,25'096,34'519,7'209,141'744,50'330,29'835,27'167,18'574,518'060,30'543,13'469,31'625,127'615,53'781,418'071,17'831,79'636,64'282,42'494,20'177,6'067,74'398,3'622,9'450,8'098,1'049'835,17'052,128'007,1'911'701,258'084,27'525,62'723,68'151,7'582,34'148,26'974,57'551,262'556,117'298,44'192,38'154,715'333,43'686,10'477,10'709
- SCORE -,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Trump,118'602,1'430'589,755'815,1'612'585,4'741'191,1'335'253,714'981,15'075,199'829,5'658'847,2'455'305,196'602,896'100,554'019,2'316'566,1'725'723,752'903,1'325'742,1'255'481,1'148'777,857'843,359'502,2'644'525,1'484'671,1'711'848,677'218,341'767,2'733'645,234'845,548'462,365'373,1'385'529,400'095,619'944,2'849'085,3'074'418,1'018'870,928'706,3'314'164,197'421,1'384'852,261'035,1'849'211,5'865'248,759'638,1'946'870,112'688,1'514'563,1'610'030,539'610,193'454
Biden,63'992,834'533,417'897,1'631'195,9'163'725,1'753'416,1'029'132,268'625,295'403,5'284'453,2'465'500,365'802,757'707,286'991,3'056'219,1'239'401,551'144,771'884,855'597,2'316'338,1'557'743,430'023,2'790'648,1'717'830,1'242'851,440'284,243'719,2'658'274,114'480,367'319,423'291,2'029'493,496'826,647'474,3'694'996,2'603'731,503'289,1'304'536,3'355'387,300'325,1'091'348,150'467,1'139'666,5'217'656,489'469,2'364'415,242'805,2'303'430,1'630'570,232'502,73'445
Jorgensen,5'198,24'902,13'039,49'406,138'187,50'469,19'739,1'532,4'977,70'046,61'888,5'525,19'556,16'401,59'791,58'703,29'466,26'204,21'636,45'782,26'301,14'176,60'287,34'994,40'932,7'309,15'194,47'340,9'347,19'805,13'066,21'459,12'446,13'014,47'802,65'069,24'686,39'367,77'453,4'976,27'908,11'095,29'816,125'273,31'220,63'784,3'573,75'641,38'415,10'462,5'765
Trump%,63.16,62.47,63.69,48.97,33.76,42.54,40.54,5.29,39.95,51.38,49.28,34.62,53.55,64.62,42.64,57.07,56.46,62.42,58.87,32.72,35.13,44.73,48.12,45.86,57.14,60.21,56.90,50.26,65.48,58.62,45.57,40.32,44.00,48.42,43.22,53.53,65.87,40.87,49.12,39.27,55.30,61.77,61.26,52.33,59.33,44.50,31.38,38.90,49.10,68.95,70.95
Biden%,34.08,36.44,35.21,49.53,65.25,55.86,58.35,94.18,59.06,47.98,49.48,64.41,45.28,33.47,56.26,40.99,41.33,36.34,40.12,65.98,63.79,53.51,50.78,53.06,41.49,39.14,40.57,48.87,31.92,39.26,52.80,59.06,54.63,50.57,56.05,45.34,32.54,57.40,49.73,59.74,43.58,35.61,37.75,46.55,38.23,54.04,67.62,59.16,49.73,29.71,26.94
Jorgensen%,2.77,1.09,1.10,1.50,0.98,1.61,1.12,0.54,0.99,0.64,1.24,0.97,1.17,1.91,1.10,1.94,2.21,1.23,1.01,1.30,1.08,1.76,1.10,1.08,1.37,0.65,2.53,0.87,2.61,2.12,1.63,0.62,1.37,1.02,0.73,1.13,1.60,1.73,1.15,0.99,1.11,2.63,0.99,1.12,2.44,1.46,1.00,1.94,1.17,1.34,2.11


In [3]:
pd.DataFrame(electors_by_state)

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
Name,Alaska,Alabama,Arkansas,Arizona,California,Colorado,Connecticut,District of Columbia,Delaware,Florida,Georgia,Hawaii,Iowa,Idaho,Illinois,Indiana,Kansas,Kentucky,Louisiana,Massachusetts,Maryland,Maine,Michigan,Minnesota,Missouri,Mississippi,Montana,North Carolina,North Dakota,Nebraska,New Hampshire,New Jersey,New Mexico,Nevada,New York,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Virginia,Vermont,Washington,Wisconsin,West Virginia,Wyoming
Electors,6,9,6,11,55,9,7,3,3,29,16,4,6,4,20,11,6,8,8,11,10,3,16,10,10,6,3,0,3,4,4,14,5,6,29,18,7,7,20,4,9,3,11,38,6,13,3,12,10,5,3


## Electors

In [4]:
pd.DataFrame(electors_by_candidate)

,Candidate,Electors
0,Trump,219
1,Biden,305


## Potential twist 

In [5]:
pd.DataFrame(electors_by_candidate_may)

,Candidate maybe,Electors
0,Biden,150
1,Trump,374


# 2. Results as of Thu 12 Nov

In [6]:
#data = load('data-cache--2020-11-09T23.58.18.360.00.00.json')
#data = load('data-cache--2020-11-11T07.57.21.383.00.00.json')
#data = load('data-cache--2020-11-12T12.27.02.803.00.00.json')
data = load('data-cache--2020-11-13T09.09.56.007.00.00.json')
new_data, new_data_short_1, electors_by_candidate, electors_by_candidate_may, electors_by_state = compute(data)
pd.DataFrame(new_data)

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,92,99,99,99,93,95,99,99,99,99,99,96,99,99,92,99,99,99,95,99,97,99,99,99,99,99,99,99,99,99,99,91,99,95,80,96,99,98,99,99,99,94,99,99,98,99,99,99,99,99,97
grand total,351'613,2'313'155,1'220'137,3'411'325,17'571'921,3'340'132,1'839'219,309'639,507'418,11'155'090,5'042'486,597'764,1'702'241,875'985,5'950'539,3'054'370,1'346'982,2'156'561,2'260'329,3'564'678,2'849'018,821'532,5'575'096,3'303'076,3'038'125,1'144'988,606'747,5'540'270,363'525,947'480,809'828,4'508'552,926'419,1'407'749,8'374'958,6'001'302,1'574'370,2'335'332,6'894'997,510'304,2'538'256,449'571,3'076'244,11'356'541,1'477'968,4'441'047,369'134,4'044'885,3'322'701,793'051,283'373
uncounted,32'494,23'131,33'386,34'113,1'404'186,200'994,25'919,5'910,7'209,141'744,50'424,29'835,27'162,18'574,517'963,30'543,13'469,31'641,127'615,53'781,105'139,17'831,79'636,64'146,42'494,20'177,6'067,74'953,3'635,9'474,8'098,428'760,17'052,86'763,1'716'602,258'084,27'525,62'723,68'949,7'582,34'148,26'974,57'551,146'715,52'623,44'410,10'068,65'395,43'686,10'477,10'709
- SCORE -,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Trump,173'233,1'430'589,755'815,1'657'250,5'536'973,1'335'253,713'778,16'306,199'829,5'658'847,2'457'920,196'602,897'140,554'019,2'316'566,1'725'723,752'903,1'326'418,1'255'481,1'148'777,926'771,359'502,2'644'525,1'485'254,1'711'848,677'218,341'767,2'744'299,235'595,550'202,365'373,1'698'349,400'095,635'089,2'871'533,3'074'418,1'018'870,928'706,3'343'692,197'421,1'384'852,261'035,1'849'211,5'865'913,840'163,1'952'834,112'688,1'556'739,1'610'030,539'610,193'454
Biden,138'070,834'533,417'897,1'668'684,10'460'073,1'753'416,1'079'223,285'728,295'403,5'284'453,2'472'083,365'802,758'352,286'991,3'056'219,1'239'401,551'144,772'285,855'597,2'316'338,1'786'733,430'023,2'790'648,1'718'683,1'242'851,440'284,243'719,2'672'900,114'902,367'919,423'291,2'353'302,496'826,671'955,3'738'403,2'603'731,503'289,1'304'536,3'403'925,300'325,1'091'348,150'467,1'139'666,5'218'631,547'976,2'379'830,242'805,2'344'336,1'630'570,232'502,73'445
Jorgensen,7'816,24'902,13'039,51'278,170'689,50'469,20'299,1'695,4'977,70'046,62'059,5'525,19'587,16'401,59'791,58'703,29'466,26'217,21'636,45'782,30'375,14'176,60'287,34'993,40'932,7'309,15'194,48'118,9'393,19'885,13'066,28'141,12'446,13'942,48'420,65'069,24'686,39'367,78'431,4'976,27'908,11'095,29'816,125'282,37'206,63'973,3'573,78'415,38'415,10'462,5'765
Trump%,54.28,62.47,63.69,49.07,34.25,42.54,39.36,5.37,39.95,51.38,49.24,34.62,53.56,64.62,42.64,57.07,56.46,62.42,58.87,32.72,33.78,44.73,48.12,45.86,57.14,60.21,56.90,50.21,65.46,58.66,45.57,41.63,44.00,48.08,43.13,53.53,65.87,40.87,48.98,39.27,55.30,61.77,61.26,52.33,58.94,44.42,31.38,39.12,49.10,68.95,70.95
Biden%,43.27,36.44,35.21,49.41,64.70,55.86,59.52,94.07,59.06,47.98,49.52,64.41,45.27,33.47,56.26,40.99,41.33,36.34,40.12,65.98,65.12,53.51,50.78,53.06,41.49,39.14,40.57,48.91,31.93,39.22,52.80,57.68,54.63,50.87,56.15,45.34,32.54,57.40,49.87,59.74,43.58,35.61,37.75,46.55,38.45,54.13,67.62,58.91,49.73,29.71,26.94
Jorgensen%,2.45,1.09,1.10,1.52,1.06,1.61,1.12,0.56,0.99,0.64,1.24,0.97,1.17,1.91,1.10,1.94,2.21,1.23,1.01,1.30,1.11,1.76,1.10,1.08,1.37,0.65,2.53,0.88,2.61,2.12,1.63,0.69,1.37,1.06,0.73,1.13,1.60,1.73,1.15,0.99,1.11,2.63,0.99,1.12,2.61,1.46,1.00,1.97,1.17,1.34,2.11


In [7]:
pd.DataFrame(electors_by_state)

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
Name,Alaska,Alabama,Arkansas,Arizona,California,Colorado,Connecticut,District of Columbia,Delaware,Florida,Georgia,Hawaii,Iowa,Idaho,Illinois,Indiana,Kansas,Kentucky,Louisiana,Massachusetts,Maryland,Maine,Michigan,Minnesota,Missouri,Mississippi,Montana,North Carolina,North Dakota,Nebraska,New Hampshire,New Jersey,New Mexico,Nevada,New York,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Virginia,Vermont,Washington,Wisconsin,West Virginia,Wyoming
Electors,6,9,6,11,55,9,7,3,3,29,16,4,6,4,20,11,6,8,8,11,10,3,16,10,10,6,3,0,3,4,4,14,5,6,29,18,7,7,20,4,9,3,11,38,6,13,3,12,10,5,3


## Electors

In [8]:
pd.DataFrame(electors_by_candidate)

,Candidate,Electors
0,Trump,219
1,Biden,305


## Potential twist 

In [9]:
pd.DataFrame(electors_by_candidate_may)

,Candidate maybe,Electors
0,Trump,311
1,Biden,213


# 3. Results as of now

In [10]:
data = load()
new_data, new_data_short_2, electors_by_candidate, electors_by_candidate_may, electors_by_state = compute(data)
pd.DataFrame(new_data)

2020-11-20T14:55:13.530+00:00


,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,99,99,99,99,98,99,99,99,100,100,99,99,99,99,99,99,99,99,100,99,99,99,99,99,99,99,99,99,100,99,99,99,99,99,84,97,100,99,99,99,100,100,99,99,99,99,100,99,99,99,100
grand total,360'449,2'326'300,1'231'162,3'419'488,17'570'370,3'288'950,1'841'270,333'938,504'010,11'066'401,5'051'095,580'271,1'703'569,876'138,6'055'250,3'061'291,1'347'012,2'156'561,2'148'062,3'584'033,3'002'828,821'532,5'598'518,3'299'855,3'055'135,1'327'073,609'880,5'567'170,359'890,959'508,811'950,4'531'192,933'058,1'419'571,8'376'930,6'098'010,1'560'699,2'388'262,6'979'770,518'789,2'513'329,422'609,3'076'298,11'426'831,1'502'512,4'485'671,365'486,4'090'294,3'322'859,802'194,275'026
uncounted,8'527,23'263,33'655,34'194,538'954,68'400,25'945,6'524,2'139,30'301,50'510,11'738,27'199,18'575,108'090,30'612,13'470,31'641,14'607,54'132,51'247,17'831,79'942,64'084,42'750,22'823,6'098,75'415,0,9'595,8'119,70'970,17'201,31'412,1'384'878,201'735,11'798,40'707,69'797,7'720,8'769,0,57'604,147'711,38'830,44'856,6'354,66'338,43'630,10'619,2'208
- SCORE -,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Trump,189'543,1'434'315,760'573,1'661'686,5'890'262,1'364'202,714'613,18'172,200'603,5'668'731,2'462'857,196'864,897'672,554'128,2'432'837,1'729'537,752'933,1'326'418,1'255'776,1'152'808,967'546,359'502,2'651'270,1'483'982,1'718'245,756'731,343'643,2'758'775,235'595,555'521,365'660,1'853'410,401'815,669'890,2'950'697,3'150'928,1'020'280,963'201,3'374'637,199'830,1'385'103,261'043,1'849'556,5'891'591,865'015,1'962'407,112'704,1'579'666,1'610'065,545'051,193'559
Biden,153'502,843'724,423'803,1'672'143,10'957'200,1'803'921,1'080'492,307'265,296'268,5'297'045,2'475'141,366'130,759'061,287'031,3'448'457,1'242'253,551'144,772'285,856'034,2'331'104,1'950'920,430'023,2'806'899,1'716'818,1'252'873,539'494,244'833,2'684'302,114'902,374'172,424'935,2'575'665,501'460,703'486,3'989'915,2'677'873,503'890,1'342'788,3'455'998,306'192,1'091'541,150'471,1'139'332,5'261'279,560'221,2'413'539,242'820,2'364'230,1'630'673,235'847,73'491
Jorgensen,8'877,24'998,13'131,51'465,183'954,52'427,20'220,1'977,5'000,70'324,62'587,5'539,19'637,16'404,65'866,58'889,29'465,26'217,21'645,45'989,33'115,14'176,60'407,34'971,41'267,8'025,15'306,48'678,9'393,20'220,13'236,31'147,12'582,14'783,51'440,67'474,24'731,41'566,79'338,5'047,27'916,11'095,29'806,126'250,38'446,64'869,3'608,80'060,38'491,10'677,5'768
Trump%,53.86,62.28,63.51,49.09,34.58,42.36,39.37,5.55,39.97,51.37,49.25,34.63,53.55,64.62,40.91,57.07,56.46,62.42,58.86,32.66,32.78,44.73,48.04,45.86,57.04,58.02,56.92,50.23,65.46,58.48,45.49,41.55,43.87,48.26,42.20,53.44,65.87,41.03,48.84,39.10,55.30,61.77,61.27,52.23,59.10,44.19,31.38,39.26,49.10,68.86,70.95
Biden%,43.62,36.64,35.39,49.39,64.34,56.01,59.52,93.85,59.03,48.00,49.50,64.40,45.28,33.47,57.98,40.99,41.33,36.34,40.12,66.04,66.10,53.51,50.86,53.06,41.59,41.36,40.55,48.88,31.93,39.39,52.86,57.75,54.75,50.68,57.06,45.42,32.53,57.20,50.01,59.91,43.58,35.61,37.74,46.65,38.27,54.35,67.61,58.75,49.73,29.79,26.94
Jorgensen%,2.52,1.09,1.10,1.52,1.08,1.63,1.11,0.60,1.00,0.64,1.25,0.97,1.17,1.91,1.11,1.94,2.21,1.23,1.01,1.30,1.12,1.76,1.09,1.08,1.37,0.62,2.54,0.89,2.61,2.13,1.65,0.70,1.37,1.06,0.74,1.14,1.60,1.77,1.15,0.99,1.11,2.63,0.99,1.12,2.63,1.46,1.00,1.99,1.17,1.35,2.11


In [11]:
pd.DataFrame(electors_by_state)

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
Name,Alaska,Alabama,Arkansas,Arizona,California,Colorado,Connecticut,District of Columbia,Delaware,Florida,Georgia,Hawaii,Iowa,Idaho,Illinois,Indiana,Kansas,Kentucky,Louisiana,Massachusetts,Maryland,Maine,Michigan,Minnesota,Missouri,Mississippi,Montana,North Carolina,North Dakota,Nebraska,New Hampshire,New Jersey,New Mexico,Nevada,New York,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Virginia,Vermont,Washington,Wisconsin,West Virginia,Wyoming
Electors,6,9,6,11,55,9,7,3,3,29,16,4,6,4,20,11,6,8,8,11,10,3,16,10,10,6,3,15,3,4,4,14,5,6,29,18,7,7,20,4,9,3,11,38,6,13,3,12,10,5,3


## Progression 

In [12]:
pd.DataFrame(dic_dict_sub(new_data_short_2, new_data_short_1))

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,7,0,0,0,5,4,0,0,1,1,0,3,0,0,7,0,0,0,5,0,2,0,0,0,0,0,0,0,1,0,0,8,0,4,4,1,1,1,0,0,1,6,0,0,1,0,1,0,0,0,3
grand total,8'836,13'145,11'025,8'163,-1'551,-51'182,2'051,24'299,-3'408,-88'689,8'609,-17'493,1'328,153,104'711,6'921,30,0,-112'267,19'355,153'810,0,23'422,-3'221,17'010,182'085,3'133,26'900,-3'635,12'028,2'122,22'640,6'639,11'822,1'972,96'708,-13'671,52'930,84'773,8'485,-24'927,-26'962,54,70'290,24'544,44'624,-3'648,45'409,158,9'143,-8'347
uncounted,-23'967,132,269,81,-865'232,-132'594,26,614,-5'070,-111'443,86,-18'097,37,1,-409'873,69,1,0,-113'008,351,-53'892,0,306,-62,256,2'646,31,462,-3'635,121,21,-357'790,149,-55'351,-331'724,-56'349,-15'727,-22'016,848,138,-25'379,-26'974,53,996,-13'793,446,-3'714,943,-56,142,-8'501
Trump,16'310,3'726,4'758,4'436,353'289,28'949,835,1'866,774,9'884,4'937,262,532,109,116'271,3'814,30,0,295,4'031,40'775,0,6'745,-1'272,6'397,79'513,1'876,14'476,0,5'319,287,155'061,1'720,34'801,79'164,76'510,1'410,34'495,30'945,2'409,251,8,345,25'678,24'852,9'573,16,22'927,35,5'441,105
Biden,15'432,9'191,5'906,3'459,497'127,50'505,1'269,21'537,865,12'592,3'058,328,709,40,392'238,2'852,0,0,437,14'766,164'187,0,16'251,-1'865,10'022,99'210,1'114,11'402,0,6'253,1'644,222'363,4'634,31'531,251'512,74'142,601,38'252,52'073,5'867,193,4,-334,42'648,12'245,33'709,15,19'894,103,3'345,46
Jorgensen,1'061,96,92,187,13'265,1'958,-79,282,23,278,528,14,50,3,6'075,186,-1,0,9,207,2'740,0,120,-22,335,716,112,560,0,335,170,3'006,136,841,3'020,2'405,45,2'199,907,71,8,0,-10,968,1'240,896,35,1'645,76,215,3
Trump%,-0.43,-0.19,-0.17,0.01,0.34,-0.18,0.00,0.18,0.02,-0.02,0.01,0.01,-0.01,0.00,-1.73,-0.00,0.00,0.00,-0.01,-0.06,-1.00,0.00,-0.08,0.01,-0.11,-2.19,0.02,0.02,0.00,-0.18,-0.08,-0.07,-0.12,0.18,-0.93,-0.09,0.00,0.16,-0.15,-0.17,0.00,0.00,0.01,-0.09,0.15,-0.23,-0.00,0.14,-0.00,-0.10,-0.00
Biden%,0.35,0.19,0.18,-0.02,-0.36,0.16,0.00,-0.23,-0.02,0.02,-0.02,-0.01,0.01,-0.00,1.73,0.00,-0.00,0.00,0.01,0.06,0.98,0.00,0.08,-0.01,0.10,2.22,-0.02,-0.03,0.00,0.17,0.07,0.07,0.12,-0.19,0.92,0.08,-0.00,-0.20,0.15,0.17,-0.00,-0.00,-0.01,0.09,-0.17,0.22,-0.01,-0.16,-0.00,0.08,0.00
Jorgensen%,0.07,-0.00,-0.00,0.00,0.02,0.02,-0.01,0.05,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,-0.00,0.00,0.00,-0.00,0.01,0.00,-0.00,0.00,0.00,-0.03,0.01,0.01,0.00,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.00,0.04,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.02,0.01,0.01,0.02,0.00,0.01,-0.00


## Progression since Sat 7 Nov

In [13]:
pd.DataFrame(dic_dict_sub(new_data_short_2, new_data_short_0))

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,49,0,0,9,32,4,0,9,1,1,0,3,0,0,7,0,0,0,5,0,13,0,0,0,0,0,0,0,1,0,0,22,0,7,6,1,1,1,0,0,1,6,0,1,6,0,8,14,0,0,3
grand total,-21'491,13'145,11'025,-239'607,-3'919'628,-51'182,52'322,14'187,-3'408,-88'689,18'072,-17'493,3'039,153,104'614,6'921,30,1'106,-112'267,19'355,142'870,0,23'422,-1'922,17'010,182'085,3'133,53'513,-2'404,14'472,2'122,44'876,6'639,11'132,-126'654,96'708,-13'671,52'930,164'615,8'485,-24'927,-26'962,54,-43'902,104'887,66'410,-31'734,-518'673,158,9'143,-8'347
uncounted,-185'621,132,269,-331'715,-6'907'941,-132'594,849,-27'995,-5'070,-111'443,180,-18'097,32,1,-409'970,69,1,16,-113'008,351,-366'824,0,306,-198,256,2'646,31,1'017,-3'622,145,21,-978'865,149,-96'595,-526'823,-56'349,-15'727,-22'016,1'646,138,-25'379,-26'974,53,-114'845,-78'468,664,-31'800,-648'995,-56,142,-8'501
Trump,70'941,3'726,4'758,49'101,1'149'071,28'949,-368,3'097,774,9'884,7'552,262,1'572,109,116'271,3'814,30,676,295,4'031,109'703,0,6'745,-689,6'397,79'513,1'876,25'130,750,7'059,287,467'881,1'720,49'946,101'612,76'510,1'410,34'495,60'473,2'409,251,8,345,26'343,105'377,15'537,16,65'103,35,5'441,105
Biden,89'510,9'191,5'906,40'948,1'793'475,50'505,51'360,38'640,865,12'592,9'641,328,1'354,40,392'238,2'852,0,401,437,14'766,393'177,0,16'251,-1'012,10'022,99'210,1'114,26'028,422,6'853,1'644,546'172,4'634,56'012,294'919,74'142,601,38'252,100'611,5'867,193,4,-334,43'623,70'752,49'124,15,60'800,103,3'345,46
Jorgensen,3'679,96,92,2'059,45'767,1'958,481,445,23,278,699,14,81,3,6'075,186,-1,13,9,207,6'814,0,120,-23,335,716,112,1'338,46,415,170,9'688,136,1'769,3'638,2'405,45,2'199,1'885,71,8,0,-10,977,7'226,1'085,35,4'419,76,215,3
Trump%,-9.30,-0.19,-0.17,0.12,0.82,-0.18,-1.17,0.26,0.02,-0.02,-0.03,0.01,-0.00,0.00,-1.73,-0.00,0.00,-0.00,-0.01,-0.06,-2.35,0.00,-0.08,0.00,-0.11,-2.19,0.02,-0.02,-0.01,-0.14,-0.08,1.24,-0.12,-0.16,-1.02,-0.09,0.00,0.16,-0.28,-0.17,0.00,0.00,0.01,-0.10,-0.23,-0.31,-0.00,0.36,-0.00,-0.10,-0.00
Biden%,9.54,0.19,0.18,-0.14,-0.92,0.16,1.17,-0.33,-0.02,0.02,0.02,-0.01,-0.00,-0.00,1.73,0.00,-0.00,0.00,0.01,0.06,2.30,0.00,0.08,-0.00,0.10,2.22,-0.02,0.01,0.01,0.13,0.07,-1.31,0.12,0.11,1.01,0.08,-0.00,-0.20,0.28,0.17,-0.00,-0.00,-0.01,0.09,0.04,0.31,-0.01,-0.41,-0.00,0.08,0.00
Jorgensen%,-0.25,-0.00,-0.00,0.02,0.10,0.02,-0.01,0.07,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,-0.00,-0.00,0.00,-0.00,0.04,0.00,-0.00,-0.00,0.00,-0.03,0.01,0.02,0.00,0.01,0.02,0.07,0.01,0.05,0.01,0.01,0.00,0.04,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.19,0.00,0.01,0.05,0.00,0.01,-0.00


## Electors

In [14]:
pd.DataFrame(electors_by_candidate)

,Candidate,Electors
0,Trump,234
1,Biden,305


## Potential twist

In [15]:
pd.DataFrame(electors_by_candidate_may)

,Candidate maybe,Electors
0,Trump,285
1,Biden,254


# 4. Electors per state

In [16]:
pd.DataFrame(electors_by_state).transpose()

,Name,Electors
AK,Alaska,6
AL,Alabama,9
AR,Arkansas,6
AZ,Arizona,11
CA,California,55
CO,Colorado,9
CT,Connecticut,7
DC,District of Columbia,3
DE,Delaware,3
FL,Florida,29
